# SQL with Python


SQL is the fundemental skill for many data science jobs. Most of the databases which support SQL are server/client based like MySQL, Postgres, SQL Server and etc. Server/client based database systems are better for more complex applications, they provide better concurrency and security measures. But on the other hand, they come with the overhead of setting up and maintaining the server and they all need a fast and reliable internet connection to interact with the server. With cloud technologies like Amazon AWS and Microsoft Azure the maintenance for the server is not an issue anymore. However, to learn SQL there should be a better and easier way. SQLite is a serverless and simple database system which supports SQL. It doesn't need fancy setup or maintenance. The whole database is stored in your local disk and everyone who has the access to the database file can open and even write into the databse. This is one of the reasons you should pay special attention for setting up file permissions to limit the access to your database. There are encryption extensions that can be used with SQLite to tighten the security measures. The purpose of this article however, is to practice basic SQL queries. Some examples are from the SQL tutorial from [KhanAcademy](https://www.khanacademy.org).

[SQLite](https://docs.python.org/2/library/sqlite3.html) is also part of Python standard modules. Any database that python can connect to, follows the DB-API guidlines specified in [PEP 0249](https://www.python.org/dev/peps/pep-0249/). This ensures a uniform set of commands for connecting to any database from Python (very cool!). This made developing high level tools like [SQLAlchemy](http://www.sqlalchemy.org/) easier resulting in a more intuitive way of working with SQL databases. 

## Create a database

One of the specification is PEP 249 is the connection object. Every database adaptor should provide a connection object which will be used to interact with the database. This can be a server based database in which case a URL and username and password will be required to create the object. For example MySQL connector is called like this:

For SQLite it can be the link to the local database file or if you want to create a new database the name of the new file.

In [1]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

#Drop the table if already exists
cur.execute("DROP TABLE IF EXISTS groceries")

# Create table
cur.execute('''CREATE TABLE groceries
                (id INT PRIMARY KEY, name TEXT, quantity INT)''')

# Insert rows of data
cur.execute("INSERT INTO groceries VALUES (0, 'Bananas', 4)")
cur.execute("INSERT INTO groceries VALUES (1, 'Peanut Butter', 1)")
cur.execute("INSERT INTO groceries VALUES (2, 'Dark Chocolate', 2)")
cur.execute("SELECT * from groceries")
print cur.fetchall()
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()


[(0, u'Bananas', 4), (1, u'Peanut Butter', 1), (2, u'Dark Chocolate', 2)]


Cool! we created a table in our database and commited the results back to the database. There is a feature in most SQL databases to automatically assign a unique id number for the records when we insert them into the database. When we specify a column as INT PRIMARY KEY, this is autoincrement in SQLite. So we can skip entering all the ids for every new item we insert into the table. 

In [12]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

#Drop the table if already exists
cur.execute("DROP TABLE IF EXISTs groceries")

# Create table
cur.execute('''CREATE TABLE groceries
                (id INTEGER PRIMARY KEY, name TEXT, quantity INT)''')

# Insert rows of data
cur.execute("INSERT INTO groceries (name, quantity) VALUES ('Bananas', 4)")
cur.execute("INSERT INTO groceries (name, quantity) VALUES ('Peanut Butter', 1)")
cur.execute("INSERT INTO groceries (name, quantity) VALUES ('Dark Chocolate', 2)")
cur.execute("SELECT * from groceries")
for row in cur.fetchall():
    print row
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()


(1, u'Bananas', 4)
(2, u'Peanut Butter', 1)
(3, u'Dark Chocolate', 2)


* <strong>Notice</strong>: SQLite offers dynamic datatype assignment. It means depending on what values you put in that columns it can assign the right type. For example you may use INT, INT8 or INTEGER as the type for a column and they all are considered INTEGER under the hood. Number of bytes can be different depending on the size of the numbers. One exception to this is the autoincrement column. You have to specify it accuratly as INTEGER if it is going to be your PRIMARY KEY. That's why we entered INTEGER in the signature above. We didn't have to do this previously since overthere we provided the ids as integers when we inserted the new items into the table. You can read about affinity in SQLite [here](http://www.tutorialspoint.com/sqlite/sqlite_data_types.htm).

Python connector provides a better way to run multiple queries. We are going to do the same example with a different more convenient syntax:

In [17]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

#Drop the table if already exists
cur.execute("DROP TABLE IF EXISTs groceries")

# Create table
cur.execute('''CREATE TABLE groceries
                (id INTEGER PRIMARY KEY, name TEXT, quantity INT)''')

# Insert rows of data
data = (('Bananas', 4),
       ('Peanut Butter', 1),
       ('Dark Chocolate', 2))

cur.executemany("INSERT INTO groceries (name, quantity) VALUES (?, ?)", data)

# select everything
cur.execute("SELECT * from groceries")
for row in cur.fetchall():
    print row
    
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()


(1, u'Bananas', 4)
(2, u'Peanut Butter', 1)
(3, u'Dark Chocolate', 2)


You can even use excute methods on the connection object without creating a cursor object. A cursor object will be created implicity. This makes it one step easier to execute queries:

In [19]:
import sqlite3
conn = sqlite3.connect('example.db')

#Drop the table if already exists
conn.execute("DROP TABLE IF EXISTs groceries")

# Create table
conn.execute('''CREATE TABLE groceries
                (id INTEGER PRIMARY KEY, name TEXT, quantity INT)''')

# Insert rows of data
data = (('Bananas', 4),
       ('Peanut Butter', 1),
       ('Dark Chocolate', 2))

conn.executemany("INSERT INTO groceries (name, quantity) VALUES (?, ?)", data)

# select everything
for row in conn.execute("SELECT * from groceries"):
    print row
    
conn.commit()
conn.close()


(1, u'Bananas', 4)
(2, u'Peanut Butter', 1)
(3, u'Dark Chocolate', 2)


We can use ":memory:" to create a database in memory. What if we don't create a unique id for items in our table. It will be generated for us (very convenient). Use ROWID column name to see the ids for each row.

In [27]:
import sqlite3
conn = sqlite3.connect(":memory:")

conn.execute("CREATE TABLE stocks (date TEXT, trans TEXT, symbol TEXT, qty REAL, price REAL)")

# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
             ('2006-04-06', 'SELL', 'GOOG', 510, 93.00),
            ]
conn.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)
for row in conn.execute("SELECT ROWID, * from stocks"):
    print row
    
conn.commit()
conn.close()

(1, u'2006-03-28', u'BUY', u'IBM', 1000.0, 45.0)
(2, u'2006-04-05', u'BUY', u'MSFT', 1000.0, 72.0)
(3, u'2006-04-06', u'SELL', u'IBM', 500.0, 53.0)
(4, u'2006-04-06', u'SELL', u'GOOG', 510.0, 93.0)


We can use WHERE and ORDER BY on columns to get the results we want.

In [2]:
import sqlite3
conn = sqlite3.connect('example.db')

#Drop the table if already exists
conn.executescript("""DROP TABLE IF EXISTS groceries;
CREATE TABLE groceries (id INTEGER PRIMARY KEY, name TEXT, quantity INTEGER, aisle INTEGER);
INSERT INTO groceries (name, quantity, aisle) VALUES ("Bananas", 4, 7);
INSERT INTO groceries (name, quantity, aisle) VALUES("Peanut Butter", 1, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Dark Chocolate Bars", 2, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Ice cream", 1, 12);
INSERT INTO groceries (name, quantity, aisle) VALUES("Cherries", 6, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Chocolate syrup", 1, 4);
""")

# select everything
for row in conn.execute("SELECT * FROM groceries ORDER BY aisle;"):
    print row

print "=========================="
for row in conn.execute("SELECT * FROM groceries WHERE aisle >= 6 ORDER BY groceries.aisle;"):
    print row
    
conn.commit()
conn.close()



(2, u'Peanut Butter', 1, 2)
(3, u'Dark Chocolate Bars', 2, 2)
(5, u'Cherries', 6, 2)
(6, u'Chocolate syrup', 1, 4)
(1, u'Bananas', 4, 7)
(4, u'Ice cream', 1, 12)
(1, u'Bananas', 4, 7)
(4, u'Ice cream', 1, 12)


Aggregating functions can be applied when selecting a column. Filtering can be done using IN and LIKE which work with WHERE clause. With LIKE we can use the following wildcards:

* The percent sign (%)

* The underscore (_)

The percent sign represents zero, one, or multiple characters. The underscore represents a single number or character. 

In [3]:
import sqlite3
conn = sqlite3.connect('example.db')

#Drop the table if already exists
conn.executescript("""DROP TABLE IF EXISTS groceries;
CREATE TABLE groceries (id INTEGER PRIMARY KEY, name TEXT, quantity INTEGER, aisle INTEGER);
INSERT INTO groceries (name, quantity, aisle) VALUES ("Bananas", 14, 7);
INSERT INTO groceries (name, quantity, aisle) VALUES("Peanut Butter", 1, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Dark Chocolate Bars", 2, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Ice cream", 1, 12);
INSERT INTO groceries (name, quantity, aisle) VALUES("Ice cream", 1, 7);
INSERT INTO groceries (name, quantity, aisle) VALUES("Cherries", 6, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Chocolate syrup", NULL, 4);
INSERT INTO groceries (name, quantity, aisle) VALUES("Chocolate shake", 23, 4);
INSERT INTO groceries (name, quantity, aisle) VALUES("Chocolate smoothie", 2, 5);
""")
for row in conn.execute("SELECT aisle, SUM(quantity) FROM groceries GROUP BY aisle"):
    print row
   
print """
Always select by the index you grouped by on otherwise 
SQL will select the first record in that category without telling you!=====================
"""
for row in conn.execute("SELECT name, AVG(aisle), SUM(quantity) FROM groceries WHERE name LIKE 'Chocolate%' OR quantity=6 GROUP BY name"):
    print row
    
conn.commit()
conn.close()

(2, 9)
(4, 23)
(5, 2)
(7, 15)
(12, 1)

Always select by the index you grouped by on otherwise 
SQL will select the first record in that category without telling you!=====================

(u'Cherries', 2.0, 6)
(u'Chocolate shake', 4.0, 23)
(u'Chocolate smoothie', 5.0, 2)
(u'Chocolate syrup', 4.0, None)


You can use HAVING to filter the results after grouping and aggregating unlike WHERE that applies before grouping.

In [4]:
import sqlite3
conn = sqlite3.connect('example.db')

#Drop the table if already exists
conn.executescript("""DROP TABLE IF EXISTS groceries;
CREATE TABLE groceries (id INTEGER PRIMARY KEY, name TEXT, quantity INTEGER, aisle INTEGER);
INSERT INTO groceries (name, quantity, aisle) VALUES ("Bananas", 14, 7);
INSERT INTO groceries (name, quantity, aisle) VALUES("Peanut Butter", 1, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Dark Chocolate Bars", 2, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Ice cream", 1, 12);
INSERT INTO groceries (name, quantity, aisle) VALUES("Ice cream", 1, 7);
INSERT INTO groceries (name, quantity, aisle) VALUES("Cherries", 6, 2);
INSERT INTO groceries (name, quantity, aisle) VALUES("Chocolate syrup", NULL, 4);
INSERT INTO groceries (name, quantity, aisle) VALUES("Chocolate shake", 23, 4);
INSERT INTO groceries (name, quantity, aisle) VALUES("Chocolate smoothie", 2, 5);
""")

for row in conn.execute("SELECT name, AVG(aisle), SUM(quantity) as total_sum FROM groceries GROUP BY name"):
    print row
    
print "========= With HAVING =============="

for row in conn.execute("SELECT name, AVG(aisle), SUM(quantity) as total_sum FROM groceries GROUP BY name HAVING total_sum > 5 "):
    print row
    
print "========= With HAVING =============="

for row in conn.execute("SELECT name, AVG(aisle), SUM(quantity) FROM groceries GROUP BY name HAVING COUNT(name) > 1 "):
    print row
    
conn.commit()
conn.close()

(u'Bananas', 7.0, 14)
(u'Cherries', 2.0, 6)
(u'Chocolate shake', 4.0, 23)
(u'Chocolate smoothie', 5.0, 2)
(u'Chocolate syrup', 4.0, None)
(u'Dark Chocolate Bars', 2.0, 2)
(u'Ice cream', 9.5, 2)
(u'Peanut Butter', 2.0, 1)
========= With HAVING ==============
(u'Bananas', 7.0, 14)
(u'Cherries', 2.0, 6)
(u'Chocolate shake', 4.0, 23)
========= With HAVING ==============
(u'Ice cream', 9.5, 2)


Using CASE statement we can add another column describing some criteria about our columns

In [50]:
import sqlite3
conn = sqlite3.connect('example.db')

#Drop the table if already exists
conn.executescript("""DROP TABLE IF EXISTS exercise_logs;
CREATE TABLE exercise_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    minutes INTEGER, 
    calories INTEGER,
    heart_rate INTEGER);

INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("biking", 30, 100, 110);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("biking", 10, 30, 105);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("dancing", 15, 200, 120);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("sex", 15, 210, 190);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("dancing", 15, 165, 120);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("tree climbing", 30, 70, 90);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("tree climbing", 25, 72, 80);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("rowing", 30, 70, 90);
INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ("hiking", 60, 80, 85);
""")

for row in conn.execute("""SELECT COUNT(*),
                                CASE 
                                    WHEN heart_rate > 220-30 THEN "above max"
                                    WHEN heart_rate > ROUND(0.90 * (220-30)) THEN "above target"
                                    WHEN heart_rate > ROUND(0.50 * (220-30)) THEN "within target"
                                    ELSE "below target"
                                END as "hr_zone"
                            FROM exercise_logs
                            GROUP BY hr_zone;
                        """):
    print row
    
print "================="
    
for row in conn.execute("""SELECT DISTINCT heart_rate FROM exercise_logs                            
                        """):
    print row
    

    
conn.commit()
conn.close()



(1, u'above target')
(4, u'below target')
(4, u'within target')
(80,)
(85,)
(90,)
(105,)
(110,)
(120,)
(190,)


In [12]:
import sqlite3
conn = sqlite3.connect('example.db')

#Drop the table if already exists
conn.execute("DROP TABLE IF EXISTS students;")
conn.execute("DROP TABLE IF EXISTS student_grades;")
conn.executescript("""CREATE TABLE students (id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT,
    birthdate TEXT);

INSERT INTO students (first_name, last_name, email, phone, birthdate)
    VALUES ("Peter", "Rabbit", "peter@rabbit.com", "555-6666", "2002-06-24");
INSERT INTO students (first_name, last_name, email, phone, birthdate)
    VALUES ("Alice", "Wonderland", "alice@wonderland.com", "555-4444", "2002-07-04");
    
CREATE TABLE student_grades (id INTEGER PRIMARY KEY,
    student_id INTEGER,
    test TEXT,
    grade INTEGER);

INSERT INTO student_grades (student_id, test, grade)
    VALUES (1, "Nutrition", 95);
INSERT INTO student_grades (student_id, test, grade)
    VALUES (2, "Nutrition", 92);
INSERT INTO student_grades (student_id, test, grade)
    VALUES (1, "Chemistry", 85);
INSERT INTO student_grades (student_id, test, grade)
    VALUES (2, "Chemistry", 95);
""")

for row in conn.execute("""SELECT * from students
                        """):
    print row
    
print "================="

for row in conn.execute("""SELECT * from student_grades
                        """):
    print row
    
print "==============="


for row in conn.execute("""SELECT * from student_grades, students
                        """):
    print row
    
print "==============="

for row in conn.execute("""SELECT students.first_name, students.last_name, student_grades.grade from student_grades JOIN students ON
students.id = student_grades.student_id
                        """):
    print row
    
print "==============="

for row in conn.execute("""SELECT students.first_name, students.last_name, student_grades.grade 
FROM student_grades LEFT OUTER JOIN students ON
students.id = student_grades.student_id
                        """):
    print row
    
conn.commit()
conn.close()

(1, u'Peter', u'Rabbit', u'peter@rabbit.com', u'555-6666', u'2002-06-24')
(2, u'Alice', u'Wonderland', u'alice@wonderland.com', u'555-4444', u'2002-07-04')
(1, 1, u'Nutrition', 95)
(2, 2, u'Nutrition', 92)
(3, 1, u'Chemistry', 85)
(4, 2, u'Chemistry', 95)
(1, 1, u'Nutrition', 95, 1, u'Peter', u'Rabbit', u'peter@rabbit.com', u'555-6666', u'2002-06-24')
(1, 1, u'Nutrition', 95, 2, u'Alice', u'Wonderland', u'alice@wonderland.com', u'555-4444', u'2002-07-04')
(2, 2, u'Nutrition', 92, 1, u'Peter', u'Rabbit', u'peter@rabbit.com', u'555-6666', u'2002-06-24')
(2, 2, u'Nutrition', 92, 2, u'Alice', u'Wonderland', u'alice@wonderland.com', u'555-4444', u'2002-07-04')
(3, 1, u'Chemistry', 85, 1, u'Peter', u'Rabbit', u'peter@rabbit.com', u'555-6666', u'2002-06-24')
(3, 1, u'Chemistry', 85, 2, u'Alice', u'Wonderland', u'alice@wonderland.com', u'555-4444', u'2002-07-04')
(4, 2, u'Chemistry', 95, 1, u'Peter', u'Rabbit', u'peter@rabbit.com', u'555-6666', u'2002-06-24')
(4, 2, u'Chemistry', 95, 2, u'Alic